In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Created a list to append all tweet attributes(data)
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:louis030195').get_items()):
    if i>100:
        break
    attributes_container.append([tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])
    
# Creating a dataframe from the tweets list above 
tweets_df = pd.DataFrame(attributes_container, columns=["Date Created", "Number of Likes", "Source of Tweet", "Tweets"])
tweets_df

In [31]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [36]:
import os
import openai
import time
import re
from IPython. display import clear_output
from tqdm import tqdm
values = open(".env", "r").read()
openai_api_key = re.findall(r"OPENAI_API_KEY=\"(.*)\"", values)[0]
openai_org = re.findall(r"OPENAI_ORG=\"(.*)\"", values)[0]
openai.api_key = openai_api_key
openai.organization = openai_org
def build_prompt(tweet):
  return f"""Tweet content: A rational person can find peace by cultivating indifference to things outside of their control.

Short list of main topics: rationality,wisdom,stoicism

###

Tweet content: Good product design requires an obsessed artist - it’s not a part-time job.

Short list of main topics: product-design,art,product

###

Tweet content: {tweet}

Short list of main topics:"""

delay = 2
embeddings = []
for k, v in tqdm(tweets_df.iterrows()):
    clear_output(wait=True)
    tweet = v["Tweets"]
    prompt = build_prompt(tweet)
    print(f"Prompt: {prompt}")
    try:
        response = openai.Completion.create(
            model="code-davinci-002",
            prompt=prompt,
            temperature=0,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["###", "\n"]
        )
    # in case of RateLimitError, wait 1 minute and try again
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(60)
        continue
    text = response['choices'][0]['text']
    if not text:
        print("Could not generate topics for tweet: ", tweet)
        time.sleep(delay)
        continue
    print(f"Completion: {text}")
    tweets_df.at[k, "Topics"] = text
    embeddings.append(model.encode([text]))
    time.sleep(delay)

Prompt: Tweet content: A rational person can find peace by cultivating indifference to things outside of their control.

Short list of main topics: rationality,wisdom,stoicism

###

Tweet content: Good product design requires an obsessed artist - it’s not a part-time job.

Short list of main topics: product-design,art,product

###

Tweet content: @pyoudeyer @FlowersINRIA @Inria @awjuliani @unity3d "Period: beg. 2018 to summer 2018" did you mean 2019 to summer 2019 ?

Short list of main topics:
Could not generate topics for tweet:  @pyoudeyer @FlowersINRIA @Inria @awjuliani @unity3d "Period: beg. 2018 to summer 2018" did you mean 2019 to summer 2019 ?


88it [14:27,  9.85s/it]


In [38]:
tweets_df

,Date Created,Number of Likes,Source of Tweet,Tweets,Topics
0,2022-10-31 10:58:34+00:00,1,Twitter Web App,@ZinaSarif Curious to know what algorithm is r...,"algorithm,running"
1,2022-10-30 16:44:11+00:00,0,Twitter Web App,I wrote a poem about linear algebra (@StableDi...,"linear-algebra,gpt3,poem"
2,2022-10-30 16:00:46+00:00,0,Twitter Web App,@fchollet Processes and culture are code for p...,"programming,culture,processes"
3,2022-10-30 15:35:47+00:00,1,Twitter Web App,@StableDiffusion &amp; @OpenAI #gpt3 in @obsdm...,"gpt3,openai,obsdmd"
4,2022-10-16 07:53:41+00:00,0,Twitter Web App,@twitnickl I really appreciate your comment Ni...,NaN
...,...,...,...,...,...
83,2019-02-19 07:53:09+00:00,1,Twitter for Android,@AndrewYNg Internet is the best school anyway,"internet,school"
84,2019-01-24 18:58:00+00:00,0,Twitter Web Client,@DeepMindAI @Blizzard_Ent @Artosis @RotterdaM0...,NaN
85,2018-12-23 16:19:15+00:00,0,Twitter for Android,@farnamstreet Siddhartha - Hermann Hesse,NaN
86,2018-12-09 19:37:05+00:00,1,Twitter for Android,@awjuliani @jeffclune I'm looking forward to s...,"open-source,algorithm,openai"
